# Линейная регрессия: прогноз оклада по описанию вакансии

Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [16]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack
import numpy as np

data_train = pandas.read_csv("salary-train.csv")
data_test = pandas.read_csv("salary-test-mini.csv")

In [17]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [18]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


Проведите предобработку:
* Приведите тексты к нижнему регистру (text.lower()).
* Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты.

In [21]:
def preprocessing(data):
    for key in data.keys():
        encoded_key = key.encode('utf-8')
        if encoded_key != 'SalaryNormalized':
            data[encoded_key] = data[encoded_key].str.lower()
        data[encoded_key] = data[encoded_key].replace('[^a-zA-Z0-9]', ' ', regex=True)

    return data

data_train = preprocessing(data_train)
data_test = preprocessing(data_test)

In [15]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355


In [22]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


* Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [4]:
vectorizer = TfidfVectorizer(min_df=5)
fd_train = vectorizer.fit_transform(data_train['FullDescription'])
fd_test = vectorizer.transform(data_test['FullDescription'])

In [24]:
print(fd_train)

  (0, 10690)	0.1996704975954175
  (0, 18017)	0.31391234621768477
  (0, 12439)	0.04910594140747719
  (0, 12142)	0.061544789163121616
  (0, 21382)	0.05147724761166183
  (0, 4349)	0.045009407862934094
  (0, 6080)	0.09107181549892054
  (0, 12553)	0.22339830733875912
  (0, 15185)	0.10506493447359962
  (0, 9624)	0.12650134137787866
  (0, 1192)	0.0438009173709425
  (0, 4403)	0.021110853545709872
  (0, 1270)	0.25755986018125293
  (0, 14410)	0.028251234898349038
  (0, 14296)	0.04651512636500756
  (0, 20595)	0.13684338693718454
  (0, 11716)	0.05368570530585719
  (0, 10249)	0.1172097452796438
  (0, 7387)	0.052010092529727776
  (0, 12546)	0.03133710594831472
  (0, 1518)	0.04311671598102828
  (0, 18372)	0.034105269355918476
  (0, 7489)	0.042482105007239805
  (0, 3095)	0.0590871311896973
  (0, 5946)	0.023247715310520525
  :	:
  (59998, 6542)	0.14362822551294233
  (59999, 1270)	0.043289934447713715
  (59999, 20830)	0.08664339188506287
  (59999, 2224)	0.04979139035909138
  (59999, 22438)	0.05028259142

In [26]:
print(fd_test)

  (0, 22611)	0.07076798658158247
  (0, 22592)	0.016591021548534805
  (0, 22526)	0.018415945966083286
  (0, 22519)	0.08730626468795558
  (0, 22438)	0.025858069885677595
  (0, 22197)	0.020397448239528073
  (0, 21690)	0.032536389439527275
  (0, 21560)	0.026408087951643407
  (0, 21420)	0.06158932893422331
  (0, 21101)	0.03730723717240812
  (0, 21026)	0.026712487972516054
  (0, 20978)	0.0702908446569574
  (0, 20830)	0.17822716123272375
  (0, 20759)	0.027699362439400063
  (0, 20704)	0.029046662459520155
  (0, 20663)	0.026624783032779514
  (0, 20644)	0.025288225748652526
  (0, 20637)	0.037955972312494514
  (0, 20595)	0.1586922234233966
  (0, 20592)	0.02014397450874452
  (0, 20361)	0.01701041256661797
  (0, 20250)	0.051652743218131146
  (0, 20181)	0.10992960914813671
  (0, 19900)	0.07160234654903495
  (0, 19897)	0.04497597657873314
  :	:
  (1, 5393)	0.0335306394775892
  (1, 5343)	0.039971685281012755
  (1, 5284)	0.07282195222082213
  (1, 4403)	0.029441389570198533
  (1, 4095)	0.055891638633594

* Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

In [5]:
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)

* Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
enc = DictVectorizer()
X_categ_train = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_categ_test = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

* Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [10]:
features_train = hstack((fd_train, X_categ_train))
X_train = hstack((features_train, np.array(data_train['SalaryNormalized'])[:,None]))

X_test = hstack((fd_test, X_categ_test))

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [11]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(features_train, data_train['SalaryNormalized'])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [12]:
predicted = clf.predict(X_test)

print(predicted)

[56555.61500155 37188.32442618]


Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.